# Le partitionnement des données

Le partitionnement (*clustering*) est une technique courante des statistiques multivariées pour effectuer des tâches de regroupement entre variables afin de révéler une structure sous-jacente. Il s’agit d’une méthode exploratoire qui aide à la classification des données en regroupant les individus dans des ensembles cohérents où la variance intra-groupes est minimisée quand la variance inter-groupes est, elle, maximisée.

Importons les modules qui seront nécessaires :

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist, squareform

## Une matrice de dissimilarité

### Avec des variables catégorielles

Prenons les réponses de cinq étudiant·es à un test comportant dix questions :

In [ ]:
n_students = 5
n_questions = 10

data = [
    [
        random.choice(['A', 'B', 'C', 'D'])
        for _ in range(n_questions)
    ]
    for _ in range(n_students)
]

df = pd.DataFrame(data, index=[f"Student {i+1}" for i in range(n_students)], columns=[f'Q{i+1}' for i in range(0, n_questions)])

display(df)

Et faisons la comparaison deux à deux pour comptabiliser le nombre de fois où leurs réponses divergent. Enfin, normalisons en divisant le résultat par le nombre de questions afin d’obtenir un score entre 0 et 1 où 0 correspond à des étudiant·es aux réponses similaires et 1 des étudiant·es qui n’auront jamais répondu pareil :

In [ ]:
data_array = df.values

# calculate differences between rows
row_diffs = (data_array[:, None] != data_array).sum(axis=2)

# normalize
dissimilarity_matrix = row_diffs / n_questions

# matrix to a DataFrame
dissimilarity_df = pd.DataFrame(dissimilarity_matrix, index=df.index, columns=df.index)

display(dissimilarity_df)

Il est à présent facile d’identifier les paires d’étudiant·es dont les réponses se ressemblent le plus :

In [ ]:
# minimum score > 0 to exclude pairs consisting of the same student
min_score = dissimilarity_df[dissimilarity_df > 0].min().min()

# all the pairs concerned
clusters = dissimilarity_df[dissimilarity_df == min_score].stack().index.tolist()

display(clusters)

### Avec des variables numériques

Dans l’exemple précédent, les variables enregistraient des données catégorielles. Si maintenant nous prenons l’exemple d’une dizaine de textes avec des scores sur 20 dans cinq catégories :

In [ ]:
# 10 texts with a score on 5 categories
n_texts = 10
categories = ["Sciences", "Politique", "Littérature", "Journalisme", "Philosophie"]

df = pd.DataFrame(
    data=np.random.randint(0, 21, size=(n_texts, len(categories))),
    index=[f"Text {i + 1}" for i in range(0, n_texts)],
    columns=categories
)

display(df)

Il n’est plus question ici de repérer les catégories où les textes ont obtenu des scores différents, aussi la première étape consiste à calculer une matrice de corrélation :

In [ ]:
correlation_matrix = df.corr()

display(correlation_matrix)

Cette matrice ressort des coefficients variant de -1 à 1 pour exprimer la corrélation entre chaque paire de variables. Pour la transformer en une matrice de dissimilarité, il suffit de calculer l’inverse de la corrélation :

In [ ]:
dissimilarity_matrix = 1 / correlation_matrix

display(dissimilarity_matrix)

Une formule alternative consiste à calculer plutôt l’opposé de la corrélation :

In [ ]:
dissimilarity_matrix = 1 - correlation_matrix

display(dissimilarity_matrix)

Puis à normaliser afin d’obtenir des scores dans l’intervalle $[0,1]$ :

In [ ]:
display(dissimilarity_matrix / 2)

De là, nous pouvons effectuer des prédictions sur les *clusters* formés entre les catégories. Peut-être la littérature et la philosophie sont-elles liées par leur coefficient de dissimilarité et, comme la philosophie et la politique sont elles-mêmes reliées, pourrions-nous en conclure que les trois disciplines forment un groupe.

**Remarque :** les données sont générées aléatoirement aussi les groupes seront-ils toujours différents.

## Une matrice de distances

Bien souvent, lorsque l’on calcule la dissimilarité entre des vecteurs, on calcule la distance euclidienne :

$$
d(\mathbf{x}, \mathbf{y}) = \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}
$$

### Calcul de la distance euclidienne

La fonction peut s’interpréter en Python :

In [ ]:
def euclidean_distance(*, a:list, b:list) -> float:
    """Euclidean distance between two vectors.
    
    Keyword arguments:
    a -- first vector
    b -- second vector
    """
    # difference between indices
    coords = [
        (x - y) ** 2
        for x, y in zip(a, b)
    ]
    # distance = square root of the sum of coords
    return sum(coords) ** .5

Pour l’appliquer à notre jeu de données, on va d’abord générer une matrice de distances nulle avant de la remplir en se servant de la symétrie :

In [ ]:
# a null matrix
pairwise_distances = np.zeros((n_texts, n_texts))

for i in range(n_texts):
    for j in range(i, n_texts):  # start at 'i' to avoid calculating b-a pair if a-b already stored
        dist = euclidean_distance(a=df.iloc[i].values.tolist(), b=df.iloc[j].values.tolist())
        pairwise_distances[i, j] = dist
        pairwise_distances[j, i] = dist  # matrix is symetric

display(pairwise_distances)

### Calcul avec *Numpy*

Calculer la distance euclidienne entre deux vecteurs *a* et *b* revient à calculer la norme du vecteur différence $a - b$.

$$
\|\mathbf{v}\| = \sqrt{\sum_{i=1}^{n} v_i^2}
$$

Or, la fonction `.linalg.norm()` de *Numpy* permet d’obtenir directement la norme d’un vecteur ; aussi pouvons-nous nous passer de la fonction `euclidean_distance()` définie plus haut :

In [ ]:
# a null matrix
pairwise_distances = np.zeros((n_texts, n_texts))

for i in range(n_texts):
    # find vector a
    vector_a = df.iloc[i].values
    for j in range(i, n_texts):
        # find vector b
        vector_b = df.iloc[j].values
        dist = np.linalg.norm(vector_a - vector_b)
        pairwise_distances[i, j] = dist
        pairwise_distances[j, i] = dist

display(pairwise_distances)

### Une bibliothèque scientifique spécialisée

La bilbiothèque *Scipy* fournit des méthodes spécialisées pour le calcul scientifique. Utilisons les fonctions `pdist` et `squareform` du module `spatial.distance` :

In [ ]:
# euclidean matrix
distances = pdist(df.values, metric='euclidean')

# to a square matrix
distance_matrix = squareform(distances)

# to a dataframe
distance_df = pd.DataFrame(distance_matrix, index=df.index, columns=df.index)

display(distance_df)

## Le regroupement hiérarchique

Jusqu’ici, les techniques de partitionnement proposées étaient plutôt simples et élémentaires : elles permettaient de dégager des appariements, tout au plus des regroupements de trois éléments. L’objectif à présent est de présenter une méthode plus systématique qui va, à chaque étape, effectuer des regroupements deux à deux jusqu’à ce qu’il ne reste plus qu’un seul groupe constitué de l’ensemble des autres.

On peut matérialiser cette méthode grâce à un dendogramme :

In [ ]:
# by default, euclidean distance
Z = hierarchy.linkage(df, method='single')

# plot dendrogram
plt.figure(figsize=(15, 7))
plt.title('Dendrogram')
plt.xlabel('Samples')
plt.ylabel('Distance')
_ = hierarchy.dendrogram(
    Z,
    labels=df.index,
    leaf_rotation=90,
    leaf_font_size=10,
    color_threshold=0.7 * max(Z[:, 2])
)
plt.show()

### Le clustering par liaison simple

Une technique assez intuitive de regroupement repose sur l’idée que clusters sont reliés si deux de leurs composants sont plus proches l’un de l’autre que de n’importe quel autre composant d’un autre cluster. Pour illustrer cela, prenons quatre clusters *A*, *B*, *C* et *D* :

|       | A      | B      | C   | D |
|-------|--------|--------|----|----|
| **A** | 0      | **12** | 13 | 34 |
| **B** | **12** | 0      | 19 | 28 |
| **C** | 13     | 19     | 0  | 15 |
| **D** | 34     | 28     | 15 | 0  |

Ici, les clusters *A* et *B* sont plus proches l’un de l’autre que de n’importe quel autre cluster, aussi pouvons-nous les regrouper. L’étape suivante va demander de réactualiser le tableau des distances où l’on remarque que les clusters $(A,B)$ et *C* sont désormais les plus proches :

|            | (A,B)  | C      | D  |
|------------|--------|--------|----|
| **(A,B)**  | 0      | **13** | 28 |
| **C**      | **13** | 0      | 15 |
| **D**      | 28     | 15     | 0  |

Après actualisation du tableau, nous obtenons :

|               | ((A,B),C) | D  |
|---------------|-----------|----|
| **((A,B),C)** | 0         | 15 |
| **D**         | 15        | 0  |

À la fin, le cluster unique que nous avons formé est : $(((A,B),C), D)$.

#### Décomposition étape par étape

L’approche du clustering par liaison simple (ou *single-linkage clustering* en anglais) est dite agglomérative (ou *bottom-up*) dans le sens où elle forme des clusters élément par élément.

À chaque étape, il est déjà possible de calculer le point de jonction entre les clusters, logiquement situé à équidistance entre leurs composants les plus proches. Par exemple, à la première étape, les clusters *A* et *B* sont à une distance de 12 l’un de l’autre. Si nous traçons une droite entre les deux et que nous devions en trouver le milieu, nous diviserions 12 par deux pour en déduire que le point de jonction entre les deux clusters se trouve à 6. À la seconde étape, le point de jonction entre les clusters *(A,B)* et *C* est situé à $13 \div 2 = 6.5$.

**Remarque :** on appellera plutôt ce point de jonction un nœud.

Ensuite, la partie cruciale consiste à réactualiser la matrice des distances selon la fonction de liaison exprimée par la formule :

$$
D(X, Y) = \min_{x \in X, y \in Y} d(x, y)
$$

Où :

- *X* et *Y* sont des clusters ;
- *x* et *y* des composants de ces clusters.

En reprenant notre exemple, et après avoir identifié que *A* et *B* étaient les clusters les plus proches et les avoir regroupés, nous devons déterminer lequel des deux était le plus proche de *C* et de *D* :

- la distance *AC* est de 13, quand *BC* est de 19, aussi considère-t-on que le cluster $(A,B)$ se situe à une distance de 13 de *C* ;
- $D((A,B),D) = \min{(D(A,D), D(B,D))} = \min{(34,28)} = 28$

#### Algorithme de résolution

Si l’on devait concevoir à la main un programme qui effectue un clustering par liaison simple, nous répertorerions les actions suivantes à mener :

- calculer la distance entre les clusters ;
- trouver les points les plus proches entre tous les clusters ;
- déterminer entre tous les clusters qui sont les plus proches.

Définissons les fonctions nécessaires :

In [ ]:
def single_linkage(k:list, l:list, m) -> float:
    """Calculate the minimum distance between clusters,
    according to the single-linkage method:
    d(k, l) = min d(u, v)

    Args:
        k -- a cluster (collection of points)
        l -- another cluster
        m -- matrix

    Usage example:
        min_distance = single_linkage([0,3], [6,2,9], m)
    """
    return min(np.linalg.norm(m[p_k] - m[p_l]) for p_k in k for p_l in l)

def points_in_cluster(cluster):
    """A cluster is a collection of points."""
    return [int(point) for point in cluster.split(',')]

def dist_between_clusters(n, clusters, m):
    return [
        0 if n == cluster else single_linkage(points_in_cluster(n), points_in_cluster(cluster), m)
        for cluster in clusters.loc[n].index
    ]

def nearest_clusters(m):
    """Find the nearest clusters in a distance matrix."""

    # looking for the minimal distance
    min_dist = m[m > 0].min().min()

    # which clusters are concerned?
    c_1, c_2 = np.where(m == min_dist)[0]

    # and their names?
    c_1_name = m.iloc[c_1].name
    c_2_name = m.iloc[c_2].name

    return (c_1_name, c_2_name)

**1e étape :** calculer une matrice des distances où au départ chaque point est un cluster.

In [ ]:
# a copy of the distance matrix
cluster_df = pd.DataFrame(distance_df.values, index=[str(i) for i in range(len(distance_df))], columns=[str(i) for i in range(len(distance_df))])

# how many clusters at the end?
n = 2

# how many steps in total?
n_steps = len(cluster_df) - n

**2e étape :** trouver les clusters les plus proches et les regrouper (à répéter autant de fois que nécessaire).

In [ ]:
for i in range(n_steps):

    # find the nearest clusters
    c_1, c_2 = nearest_clusters(cluster_df)
    new_cluster = ','.join([c_1, c_2])

    # new cluster in town
    cluster_df.loc[new_cluster] = np.zeros(len(cluster_df))
    cluster_df[new_cluster] = np.zeros(len(cluster_df))

    # update distance matrix
    cluster_df.loc[new_cluster] = dist_between_clusters(new_cluster, cluster_df, distance_df.values)
    cluster_df[new_cluster] = dist_between_clusters(new_cluster, cluster_df, distance_df.values)

    # delete merged clusters
    cluster_df.drop([c_1, c_2], axis=0, inplace=True)
    cluster_df.drop([c_1, c_2], axis=1, inplace=True)

    # result
    print(f"Step {i + 1}: {','.join(str(int(x) + 1) for x in new_cluster.split(','))}")

### Le clustering par liaison complète

Autre technique de partitionnement hiérarchique, le clustering par liaison complète (ou *complete-linkage clustering* en anglais) ne va pas considérer, contrairement au clustering par liaison simple, que la distance des points les plus proches de deux clusters définisse la distance entre les clusters, mais que ce soit plutôt la distance des points les plus éloignés.

La fonction de laison s’exprime désormais par l’expression mathématique :

$$
D(X, Y) = \max_{x \in X, y \in Y} d(x, y)
$$

En reprenant l’exemple plus haut, à la 1e étape les clusters *A* et *B* sont toujours les plus proches :

|       | A      | B      | C   | D |
|-------|--------|--------|----|----|
| **A** | 0      | **12** | 13 | 34 |
| **B** | **12** | 0      | 19 | 28 |
| **C** | 13     | 19     | 0  | 15 |
| **D** | 34     | 28     | 15 | 0  |

Après réactualisation du tableau des distances avec la fonction de laison complète, nous obtenons :

|            | (A,B) | C      | D      |
|------------|-------|--------|--------|
| **(A,B)**  | 0     | 19     | 34     |
| **C**      | 19    | 0      | **15** |
| **D**      | 34    | **15** | 0      |

Avec cette technique, le cluster $(A,B)$ s’est éloigné de *C* et de *D*, si bien que ces deux derniers sont désormais les plus proches l’un de l’autre :

|               | (A,B) | (C,D) |
|---------------|-------|-------|
| **(A,B)**     | 0     | 34    |
| **(C,D)**     | 34    | 0     |

À la fin, le cluster unique que nous avons formé est : $((A,B), (C,D))$.

#### Décomposition par étapes

À chaque étape, il convient de regrouper d’abord deux clusters en fonction de leur proximité puis de recalculer la distance qui les sépare des autres en prenant le point le plus éloigné.

Ainsi, après avoir identifié dans notre exemple que *A* et *B* étaient les clusters les plus proches et les avoir regroupés, nous devons déterminer lequel des deux était le plus **éloigné** de *C* et de *D* :

- la distance *AC* est de 13, quand *BC* est de 19, aussi considère-t-on que le cluster $(A,B)$ se situe à une distance de 19 de *C* ;
- $D((A,B),D) = \max{(D(A,D), D(B,D))} = \max{(34,28)} = 34$

#### Algorithme de résolution

L’algorithme est pour ainsi dire identique à celui du clustering par liaison simple. Nous avons juste besoin d’une fonction spécifique :

In [ ]:
def complete_linkage(k:list, l:list, m) -> float:
    """Calculate the maximum distance between clusters,
    according to the complete-linkage method:
    d(k, l) = max d(u, v)

    Args:
        k -- a cluster (collection of points)
        l -- another cluster
        m -- matrix

    Usage example:
        max_distance = complete_linkage([0,3], [6,2,9], m)
    """
    return max(np.linalg.norm(m[p_k] - m[p_l]) for p_k in k for p_l in l)

Et de redéfinir `dist_between_clusters()` en faisant appel à la fonction `complete_linkage()` :

In [ ]:
def dist_between_clusters(n, clusters, m):
    return [
        0 if n == cluster else complete_linkage(points_in_cluster(n), points_in_cluster(cluster), m)
        for cluster in clusters.loc[n].index
    ]

Pour le reste, la procédure est similaire :

In [ ]:
# a copy of the distance matrix
cluster_df = pd.DataFrame(distance_df.values, index=[str(i) for i in range(len(distance_df))], columns=[str(i) for i in range(len(distance_df))])

# how many clusters at the end?
n = 3

# how many steps in total?
n_steps = len(cluster_df) - n

In [ ]:
for i in range(n_steps):

    # find the nearest clusters
    c_1, c_2 = nearest_clusters(cluster_df)
    new_cluster = ','.join([c_1, c_2])

    # new cluster in town
    cluster_df.loc[new_cluster] = np.zeros(len(cluster_df))
    cluster_df[new_cluster] = np.zeros(len(cluster_df))

    # update distance matrix
    cluster_df.loc[new_cluster] = dist_between_clusters(new_cluster, cluster_df, distance_df.values)
    cluster_df[new_cluster] = dist_between_clusters(new_cluster, cluster_df, distance_df.values)

    # delete merged clusters
    cluster_df.drop([c_1, c_2], axis=0, inplace=True)
    cluster_df.drop([c_1, c_2], axis=1, inplace=True)

    # result
    print(f"Step {i + 1}: {','.join(str(int(x) + 1) for x in new_cluster.split(','))}")